In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
caffe_root = '/home/stephanos/Development/caffe-master/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')
import pickle
import caffe
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
# Load the R-CNN model from disk...

caffe.set_device(0)  # if we have multiple GPUs, pick the first one
caffe.set_mode_gpu()

rcnn_model_def = caffe_root + "models/bvlc_reference_rcnn_ilsvrc13/deploy.prototxt"
rcnn_model_weights = caffe_root + 'models/bvlc_reference_rcnn_ilsvrc13/bvlc_reference_rcnn_ilsvrc13.caffemodel'

rcnn_net = caffe.Net(rcnn_model_def, rcnn_model_weights, caffe.TEST)


# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print('mean-subtracted values:', zip('BGR', mu))

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': rcnn_net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR



# set the size of the input (we can skip this if we're happy
#  with the default; we can also change it later, e.g., for different batch sizes)
rcnn_net.blobs['data'].reshape(50,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227

# load ML models
gb = pickle.load(open('gsgbFinalv1','rb'))
countvect = pickle.load(open('CountVectV1','rb'))


def getPrice(imagePath, title):    
    # transform the image and copy it into the net
    image = caffe.io.load_image(imagePath)

    height = np.size(image, 0)
    width = np.size(image, 1)
    ratio = height / width
    
    if height >= width:
        height = 227
        width = height / ratio
    else:
        width = 227
        height = width * ratio
    
    image = caffe.io.resize_image( image, (int(height),int(width)), interp_order=3 )
    
    rcnn_net.blobs['data'].data[...] = transformer.preprocess('data', image)

    # perform classification
    output = rcnn_net.forward()
    
    # Get the features:
    features = rcnn_net.blobs.get('fc7').data[0].tolist()
    
    #show the image
    plt.imshow(image)
    
    #get the price
    return gb.predict(features + countvect.transform([title]).toarray()[0])[0]

mean-subtracted values: <zip object at 0x7fa61c97e548>


In [3]:
test_path = "/home/stephanos/Downloads/courses/HackOnData-2017/Amazon/baby/images/0188399313|41SwthpdD9L._SX300_.jpg"
getPrice(test_path, 'brown stroller')

/home/stephanos/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/stephanos/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[0.0,
 1.5750726461410522,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.2756216526031494,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.843724250793457,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6563944220542908,
 0.15510904788970947,
 0.0,
 0.18699097633361816,
 0.0,
 0.1601862907409668,
 0.0,
 0.0,
 0.0,
 0.0,
 2.20346736907959,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.336296319961548,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.6893115043640137,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.023578405380249,
 0.0,
 0.0,
 0.0,
 0.0,
 0.7060699462890625,
 0.7504470944404602,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.10006523132324219,
 0.0,
 0.7782105207443237,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.24755114316940308,
 0.0,
 0.0,
 0.0,